In [1]:
import os
import time
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import pickle as pkl
import datetime as dt
import tensorflow as tf
from matplotlib import style
import yahoo_finance as yahoo
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.finance import candlestick_ohlc
from IPython.display import display, Math, Latex
style.use('ggplot')

C:\Users\User\Anaconda3\envs\py35\lib\site-packages\matplotlib\cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [5]:
getter = yahoo.Share('^IXCO')
ixco = getter.get_historical('2000-1-1', '2016-12-31');
ixco_date = np.array([ixco[i]['Date'] for i in range(len(ixco))])
ixco_date = np.flip(ixco_date, axis=0)
ixco_close = np.array([ixco[i]['Close'] for i in range(len(ixco))])
ixco_close = np.flip(ixco_close, axis=0)
ixco_close = ixco_close.astype(float)
pd.to_datetime(ixco_date)

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14',
               ...
               '2016-12-16', '2016-12-19', '2016-12-20', '2016-12-21',
               '2016-12-22', '2016-12-23', '2016-12-27', '2016-12-28',
               '2016-12-29', '2016-12-30'],
              dtype='datetime64[ns]', length=4277, freq=None)

In [6]:
ibm = pd.read_csv('./ibm/ibm.csv')
ibm['Date'] = pd.to_datetime(ibm['Date'])
ibm.set_index('Date', inplace=True)
ibm = ibm.loc['2000-01-01':'2016-12-31']
ibm.set_index(pd.to_datetime(ixco_date), inplace=True)
ibm['IXCO'] = pd.Series(ixco_close, index=ibm.index)
ibm.head()

,Open,High,Low,Close,Volume,Adj Close,IXCO
2000-01-03,112.4375,116.0000,111.875,116.0000,10347700,87.761136,2368.820068
2000-01-04,114.0000,114.5000,110.875,112.0625,8227800,84.782175,2234.530029
2000-01-05,112.9375,119.7500,112.125,116.0000,12733200,87.761136,2220.600098
2000-01-06,118.0000,118.9375,113.500,114.0000,7971900,86.248013,2113.300049
2000-01-07,117.2500,117.9375,110.625,113.5000,11856700,85.869732,2204.379883


In [7]:
year = {
    2000: ibm.loc['2000-01-01':'2000-12-31'].index,
    2001: ibm.loc['2001-01-01':'2001-12-31'].index,
    2002: ibm.loc['2002-01-01':'2002-12-31'].index,
    2003: ibm.loc['2003-01-01':'2003-12-31'].index,
    2004: ibm.loc['2004-01-01':'2004-12-31'].index,
    2005: ibm.loc['2005-01-01':'2005-12-31'].index,
    2006: ibm.loc['2006-01-01':'2006-12-31'].index,
    2007: ibm.loc['2007-01-01':'2007-12-31'].index,
    2008: ibm.loc['2008-01-01':'2008-12-31'].index,
    2009: ibm.loc['2009-01-01':'2009-12-31'].index,
    2010: ibm.loc['2010-01-01':'2010-12-31'].index,
    2011: ibm.loc['2011-01-01':'2011-12-31'].index,
    2012: ibm.loc['2012-01-01':'2012-12-31'].index,
    2013: ibm.loc['2013-01-01':'2013-12-31'].index,
    2014: ibm.loc['2014-01-01':'2014-12-31'].index,
    2015: ibm.loc['2015-01-01':'2015-12-31'].index,
    2016: ibm.loc['2016-01-01':'2016-12-31'].index,
}

In [17]:
ibm['ewma_5'] = ibm['Close'].ewm(span=5).mean()
ibm['ewma_22'] = ibm['Close'].ewm(span=22).mean()
ibm['ewma_200'] = ibm['Close'].ewm(span=200).mean()
ibm['MACD'] = ibm['Close'].ewm(span=12).mean() - ibm['Close'].ewm(span=26).mean()
ibm.head()

,Open,High,Low,Close,Volume,Adj Close,IXCO,ewma_5,ewma_22,ewma_200,Lower band,Upper band,Middle band,MACD
2000-01-03,112.4375,116.0000,111.875,116.0000,10347700,87.761136,2368.820068,116.000000,116.000000,116.000000,NaN,NaN,NaN,0.000000
2000-01-04,114.0000,114.5000,110.875,112.0625,8227800,84.782175,2234.530029,113.637500,113.941761,114.021406,NaN,NaN,NaN,-0.088341
2000-01-05,112.9375,119.7500,112.125,116.0000,12733200,87.761136,2220.600098,114.756579,114.691113,114.687544,NaN,NaN,NaN,0.009537
2000-01-06,118.0000,118.9375,113.500,114.0000,7971900,86.248013,2113.300049,114.442308,114.494094,114.513071,NaN,NaN,NaN,-0.018389
2000-01-07,117.2500,117.9375,110.625,113.5000,11856700,85.869732,2204.379883,114.080569,114.257564,114.306384,NaN,NaN,NaN,-0.053072


In [18]:
ibm['Lower band'] = ibm['Close'].rolling(window=20).mean() - 2*ibm['Close'].rolling(window=20).std()
ibm['Upper band'] = ibm['Close'].rolling(window=20).mean() + 2*ibm['Close'].rolling(window=20).std()
ibm['Middle band'] = ibm['Close'].rolling(window=20).mean()
ibm.head()

,Open,High,Low,Close,Volume,Adj Close,IXCO,ewma_5,ewma_22,ewma_200,Lower band,Upper band,Middle band,MACD
2000-01-03,112.4375,116.0000,111.875,116.0000,10347700,87.761136,2368.820068,116.000000,116.000000,116.000000,NaN,NaN,NaN,0.000000
2000-01-04,114.0000,114.5000,110.875,112.0625,8227800,84.782175,2234.530029,113.637500,113.941761,114.021406,NaN,NaN,NaN,-0.088341
2000-01-05,112.9375,119.7500,112.125,116.0000,12733200,87.761136,2220.600098,114.756579,114.691113,114.687544,NaN,NaN,NaN,0.009537
2000-01-06,118.0000,118.9375,113.500,114.0000,7971900,86.248013,2113.300049,114.442308,114.494094,114.513071,NaN,NaN,NaN,-0.018389
2000-01-07,117.2500,117.9375,110.625,113.5000,11856700,85.869732,2204.379883,114.080569,114.257564,114.306384,NaN,NaN,NaN,-0.053072


In [20]:
ibm['AO'] = ((ibm['High'] + ibm['Low'])/2).rolling(window=5).mean() - ((ibm['High'] + ibm['Low'])/2).rolling(window=34).mean()
ibm['AC'] =  ibm['AO'] - ibm['AO'].rolling(window=5).mean()
ibm.head()

,Open,High,Low,Close,Volume,Adj Close,IXCO,ewma_5,ewma_22,ewma_200,Lower band,Upper band,Middle band,MACD,AO,AC
2000-01-03,112.4375,116.0000,111.875,116.0000,10347700,87.761136,2368.820068,116.000000,116.000000,116.000000,NaN,NaN,NaN,0.000000,NaN,NaN
2000-01-04,114.0000,114.5000,110.875,112.0625,8227800,84.782175,2234.530029,113.637500,113.941761,114.021406,NaN,NaN,NaN,-0.088341,NaN,NaN
2000-01-05,112.9375,119.7500,112.125,116.0000,12733200,87.761136,2220.600098,114.756579,114.691113,114.687544,NaN,NaN,NaN,0.009537,NaN,NaN
2000-01-06,118.0000,118.9375,113.500,114.0000,7971900,86.248013,2113.300049,114.442308,114.494094,114.513071,NaN,NaN,NaN,-0.018389,NaN,NaN
2000-01-07,117.2500,117.9375,110.625,113.5000,11856700,85.869732,2204.379883,114.080569,114.257564,114.306384,NaN,NaN,NaN,-0.053072,NaN,NaN


In [22]:
ibm['%R'] = (ibm['High'].max() - ibm['Close'])*100/(ibm['High'].max()-ibm['Low'].min())
ibm.head()

,Open,High,Low,Close,Volume,Adj Close,IXCO,ewma_5,ewma_22,ewma_200,Lower band,Upper band,Middle band,MACD,AO,AC,%R
2000-01-03,112.4375,116.0000,111.875,116.0000,10347700,87.761136,2368.820068,116.000000,116.000000,116.000000,NaN,NaN,NaN,0.000000,NaN,NaN,61.708565
2000-01-04,114.0000,114.5000,110.875,112.0625,8227800,84.782175,2234.530029,113.637500,113.941761,114.021406,NaN,NaN,NaN,-0.088341,NaN,NaN,64.140772
2000-01-05,112.9375,119.7500,112.125,116.0000,12733200,87.761136,2220.600098,114.756579,114.691113,114.687544,NaN,NaN,NaN,0.009537,NaN,NaN,61.708565
2000-01-06,118.0000,118.9375,113.500,114.0000,7971900,86.248013,2113.300049,114.442308,114.494094,114.513071,NaN,NaN,NaN,-0.018389,NaN,NaN,62.943972
2000-01-07,117.2500,117.9375,110.625,113.5000,11856700,85.869732,2204.379883,114.080569,114.257564,114.306384,NaN,NaN,NaN,-0.053072,NaN,NaN,63.252824


In [ ]:
ibm_train = ibm.dropna(axis=0, how='any')
ibm_train_x = ibm_train.drop(['Close'])